In [12]:
import pandas as pd
import numpy as np

In [13]:
def signal_coils_summary_no_buckets(df, signal_cols, coil_id_col='Coil_ID', output_csv='withoutbucket_coil_signal_summary_2std.csv'):
    results = []
    
    num_coils = df.shape[0]
    
    for signal in signal_cols:
        signal_values = df[signal].dropna()
        if signal_values.empty:
            continue
        
        mean_val = signal_values.mean()
        std_val = signal_values.std()
        lower_bound = mean_val - 2 * std_val
        upper_bound = mean_val + 2 * std_val
        
        within_range = signal_values[(signal_values >= lower_bound) & (signal_values <= upper_bound)]
        outside_range = signal_values[(signal_values < lower_bound) | (signal_values > upper_bound)]
        
        results.append({
            'No_of_Coils': num_coils,
            'No_of_Coils_within_range': within_range.count(),
            'No_of_Coils_outside_range': outside_range.count(),
            'Signal_Name': signal,
            'Signal_mean': mean_val,
            'Signal_std': std_val
        })
    
    summary_df = pd.DataFrame(results)
    summary_df.to_csv(output_csv, index=False)
    print(f"Summary saved to {output_csv}")


In [14]:
df = pd.read_csv("bucket_coil_signal_averages_allsignals.csv")
# List signal columns (exclude ID and bucket columns)
signals = [col for col in df.columns if col not in ['coil_id', 'Bucket_ID', 'Bucket_Name', 'CM_WIDTH', 'CP_X4GAUGE', 'Width_Bin', 'Gauge_Bin']]
signal_coils_summary_no_buckets(df, signal_cols=signals, coil_id_col='coil_id')

Summary saved to withoutbucket_coil_signal_summary_2std.csv


## check high volatile signals (>5% coils outside range)

In [15]:
def save_unique_signals(csv_path, output_csv='withoutBucket_Signals_with_high_volatility_2std.csv', threshold=0.05):
    df = pd.read_csv(csv_path)
    df['Outside_Percent'] = df['No_of_Coils_outside_range'] / df['No_of_Coils']
    filtered = df[df['Outside_Percent'] > threshold]
    
    # Get unique signal names
    unique_signals = filtered['Signal_Name'].drop_duplicates()
    
    # Save unique signals as CSV
    unique_signals.to_csv(output_csv, index=False, header=['Signal_Name'])
    print(f"Unique signals saved to {output_csv}")
    return unique_signals

In [16]:
signals_outside_5_percent = save_unique_signals('withoutbucket_coil_signal_summary_2std.csv', threshold=0.05)

Unique signals saved to withoutBucket_Signals_with_high_volatility_2std.csv
